# Processing the data
이전 chapter에 이어서, 이번엔 특정 task를 위한 classifier를 학습시키는 방법에 대해 알아보자!


In [11]:
#same as before
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
#new!
batch["labels"] = torch.tensor([1,1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

과정은 다음과 같지만, fine-tune을 위해선 더 큰 데이셋이 필요!   
이 section에서는 MRPC(Microsoft Research Paraphrase Corpus) dataset을 사용하여 fine-tune을 할 예정이다.   
MRPC dataset은 5801쌍의 문장으로 구성되어, 두 문장이 같은 의미인지를 파악하는 테스크에 대한 데이터셋이며, 이 데이터셋은 크기가 작아 실습에 적절하기에 선택하였다고 함.

## Loading a dataset from the Hub
- hub에는 모델뿐만 아니라 여러 언어로 된 데이터셋도 존재
- [dataset](https://huggingface.co/datasets), 여기에서 로드 후 처리하는 것이 좋음
- MPRC는 GLUE benchmark 중 하나의 데이터셋으로, 10개의 서로 다른 텍스트 분류 작업에서 ML 성능을 측정하는데 사용!

In [16]:
from datasets import load_dataset
raw_datasets = load_dataset("glue","mrpc")
raw_datasets

Downloading: 28.8kB [00:00, 5.78MB/s]                   
Downloading: 28.7kB [00:00, 14.3MB/s]                   


Downloading: 6.22kB [00:00, 3.13MB/s]
Downloading: 1.05MB [00:00, 1.47MB/s]
Downloading: 441kB [00:00, 794kB/s]
100%|██████████| 3/3 [00:06<00:00,  2.05s/it]


Dataset glue downloaded and prepared to C:\Users\ChangYong\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 81.11it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

mprc datasetdict을 보면 
- 3개 set : training, validation, test
- 각 set 구성 : sen1, sen2, label, idx
- 각 set은 num_rows 만큼 sent-pairs가 존재

In [30]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

각 row별로 정수가 부여되어 있고, idx가 row의 정수번호이다   
dataset features는 다음과 같다   
classLabel이 0이면 not_equivalent, 1이면 equivalent

In [31]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'idx': Value(dtype='int32', id=None)}

## Preprocessing a dataset
- 데이터셋 전처리는, 기존과 같이 텍스트를 모델 inputs에 맞게, 숫자로 변환해야 함
- 문장 쌍의 첫번째와 두번를 다음과 같이 직접 토큰화 가능

In [38]:
tokenized_sent1 = tokenizer(raw_datasets["train"]["sentence1"], padding=True, truncation=True, return_tensors="pt")
tokenized_sent2 = tokenizer(raw_datasets["train"]["sentence2"], padding=True, truncation=True, return_tensors="pt")

하지만 이렇게 처리를 하면, 두 문장이 관련 있는지 없는지를 예측할 수 없음   
두 시퀀스를 쌍으로 처리하고 적절한 전처리를 해야 함   
다행히, 토크나이저는 한 쌍의 시퀀스를 처리가능

input_type_ids는 해당 문장이 첫번째 seq인지 두번째seq인지를 구분하는 값!

다만, checkpoint에 따라 token_type_ids 는 불필요 할 수도 있음
- e.g.) distilBERT => They are only returned when the model will know what to do with them, because it has seen them during its pretraining.

일반적으로 model과 tokenizer는 같은 checkpoint를 사용하기에  부분은 고려안해도 됨

In [45]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
print(inputs)

print("\n",tokenizer.convert_ids_to_tokens(inputs["input_ids"]))

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

 ['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


이제 토크나이저가 한 쌍의 문장을 어떻게 다루는지 보았으므로, 실제 tokenized_dataset을 생성해보자

In [ ]:
tokenized_dataset = tokenizer(raw_datasets["train"]["sentence1"],
                              raw_datasets["train"]["sentence2"],
                              padding=True,
                              truncation=True,
                              return_tensors="pt")

- 이 방법도 잘 잘독하지만, 메모리 리소스 제한으로 인해 쉽지 않음...   
- data를 dataset 형태로 유지하기 위해 dataset.map 방법을 사용해볼 예정
- 이 방법은 tokenization뿐만 아니라 더 많은 전처리 task도 수행할 수 있는 만큼 작업의 유연성이 증가됨!
우선 map함수를 위해 tokenize function을 정의해보자

In [46]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

- 이 함수는 dataset의 items와 같이 dictionary에서 값을 받아서 input_ids, attention_mask, token_type_ids이 key로 된 값들을 반환!
- 그리고 map 함수를 쓸 때 bathched=True를 사용하면 tokenization 속도를 높일 수 있음
- 그리고 현재 토크나이저에 padding을 제외했는데, 이는 배치별로 패디을 진행하여 최대 길이가 아니라 배치의 최대 길이로 패딩을 진행
- 이렇게 하면 배치별로 길이가 매우 다르기에, 많은 시간과 소스를 절약할 수 있음!
- 단, 만능은 아님. 만약 seq 길이별 비슷한 의미를 가지고 있다고 가정한다면, 각 배치별로는 비슷한 의미끼리 묶이지만, 다른 배치와는 의미가 다를 수 있음. 따라서 이 경우 모델 성능 저하를 유발 할 수 있음!
- 여하튼, Batched=True를 사용하여 batched별로 함수가 적용됨

In [49]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

100%|██████████| 4/4 [00:00<00:00, 16.46ba/s]
Loading cached processed dataset at C:\Users\ChangYong\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-54bdf54eb2bd302f.arrow
Loading cached processed dataset at C:\Users\ChangYong\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-83846288736966a5.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

- Num_proc argument로 process 개수를 지정 시 multiprocessing이 가능!
- 다만, 해당 토크나이저 라이브러리에서는 이미 빠른 샘플 처리를 위해 여러 스레드를 사용
- 따라서 이 라이브러리에서 지원하는 fast tokenizer를 사용하지 않을 경우 이 옵션을 사용하면 보다 빠른 처리가 가능
---
- tokenize_function은 input_ids, attention_mask, token_type_ids가 있는 dict를 반환
- 앞으로 진행해볼, 위에서 말한 것처럼, batch별로 패딩을 하는 것을 dynamic padding이라고 함

## Dynamic padding
- PyTorch에서는 배치 내의 data sample을 전처리하는 collate 함수가 있음
- 이것은 dataloader를 만들 때 전달할 수 있는 argument
- default는 샘플들을 torch tensor로 변환하고 연결해주는 역할
- 다만 여기선, 입력값의 크기가 모두 같지 않기에 사용안함
- 패딩을 위한 collate 함수는 transformers 라이브러리에서 지원!!
  - **DataCollatorWithPadding**

In [101]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

데스트를 위해 training set 일부만 가져와서 살펴보자

In [114]:
samples = tokenized_datasets["train"][:8]
samples = {k:v for k, v in samples.items()\
           if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

samples의 seq 길이는 다양하며, 동적 패딩이 없으면 모두 seq_len=67만큼 padding이 되지만, 동적패딩을 하면 batch별 최대 길이로 패딩이 진행된다. data_collator에서 배치에 동적으로 패딩되는지 다시 확인해보자

In [115]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 67]),
 'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'labels': torch.Size([8])}

## Fine-tuning a model with the Trainer API
- transformers에서는 fine-tune을 위한 traniner class를 지원!!
- 데이터 전처리를 끝냈다면, Trainer를 define하기 위해 몇 가지만 남은 상태!!

이번 section에선 지난 section을 이어서 실행시킬 것!!

In [161]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Reusing dataset glue (C:\Users\ChangYong\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)

100%|██████████| 3/3 [00:00<00:00, 599.96it/s]
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ChangYong/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute"

### Training
- Trainer를 define하기 위해 남은 첫번째는 학습 및 평가에 활용할 하이퍼파라미터가 포함된 TrainingArguments를 define하는 것!
- 지정할 argument는 모델이 저장될 디렉토리와 도중에 저장될 checkpoint의 위치!!
- 나머지는 기본으로 둘 것이며, 이게 기본적인 fine-tuning에 매우 적합하다고 함


In [162]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [163]:
training_args

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=test-trainer\runs\Sep27_00-39-41_DESKTOP-FJL0CIS,
logging_first_step=False,
logging_steps=500,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerType.LINEAR,
max_grad_norm=1.0,
max_steps=-1,
metric_fo

다음 step은 모델을 정의

In [164]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ChangYong/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bi

- 아래 경고문은, BERT는 문장 쌍에 대해 pre-trained 되어 있지 않으므로 classifier가 새로운 것으로 지정되는 것
- 그리고 그에 따라 일부 가중치도 초기화가 되므로, down-stream task를 위해 추가로 학습을 권장하는 내용

이제 모델이 확보되었으면, model, training_args, training & validation datsets, data_collator, tokenizer를 가지고 Trainer를 정의하자

In [165]:
from transformers import Trainer
trainer = Trainer(
    model, #model
    training_args, #Trainer의 arguments
    train_dataset = tokenized_datasets["train"], #토큰화만 수행된 데이터셋
    eval_dataset = tokenized_datasets["validation"], #토큰화만 수행된 데이터셋
    data_collator=data_collator, #batch별 padding
    tokenizer=tokenizer, #토크나이저
)

이제 fine-tune은 trainer로 학습을 진행하면 끝!

In [ ]:
trainer.train()

- 이렇게 하면 finetune이 시작되고 500 step마다 loss에 대한 log값을 출력해줌
- 그러나 다음과 같은 이유로 모델이 잘 수행되는지 확인이 불가
  - Trainer를 선언할 때 evaluation_strategy를 epoch or steps으로 지정을 안함
  - evaluation을 위한 compute_metrics를 제공하지 않음!


## Evaluation
- 다시,!! compute_metrics 함수를 구현하고 trainer에 어떻게 사용할지 알아보자
- 우선 compute_metrics은 trainer의 EvalPrediction object를 사용

In [167]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Prediction *****
  Num examples = 408
  Batch size = 8








































































































(408, 2) (408,)


- predict의 output은 predictions, label_ids, metrics로 된 name_tuple
- metrics는 loss와 runtime이 포함되어 있음
- compute_metrics에 predictions을 포함하면 반환되는 metric의 하나로 포함됨!
- 보다시피 예측은 (408,2), label은 (408,)로 (408,2)는 logits이며 비교할 수 있는 값으로 변환을 위해선 두번째 축의 최대값이 있는 인덱스로 반환하여 그 인덱스를 사용!
- label은 원래 데이터셋의 label 그대로임

In [168]:
import numpy as np
preds = np.argmax(predictions.predictions,axis=-1)


- 이제 preds와 label 비교가 가능
- compute_metric 함수를 구현하기 위해 load_metric함수를 사용하여 MPRC 데이터세트와 관련 메트릭을 가져와보자

In [169]:
from datasets import load_metric

metric = load_metric("glue","mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.6764705882352942, 'f1': 0.7836065573770491}

- classifier가 랜덤으로 초기화 되므로 결과가 조금 달라질 수 있음
- metric으로는 accuracy & f1

이제 compute_metrics를 구해보자

In [170]:
def compute_metrics(eval_preds):
    metric = load_metric("glue","mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions,references=labels)

- compute_metrics 함수를 이제 Trainer의 argument로 넣자!
- 단 traning_args도 다시 선언하며, 이 때 eval_stregy를 epoch로 설정



In [171]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model= AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

trainer=Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ChangYong/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_he

In [ ]:
trainer.train()

# A full training
- 이번엔 Trainer를 사용하지 않고 지난 섹션과 동일한 결과는 얻는 방법에 대해 공부해보자
- 섹션 2의 데이터 처리를 완료했다는 가정 하에 진행하며, 다음은 필요한 모든 내용에 대한 코드이다!!

In [182]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at C:\Users\ChangYong\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-2ff2f76fa35622cd.arrow
Loading cached processed dataset at C:\Users\ChangYong\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-32e225bb550d8c33.arrow
Loading cached processed dataset at C:\Users\ChangYong\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-36450eac73a2f315.arrow


## Prepare for training
- training을 위해선 몇가지 object를 정의해야 함
- 첫번째는 dataloader, 다만 그 전에 토큰화된 데이터셋에 후처리를 조금 해야함!
  - 모형이 사용하지 않는 값에 해당하는 열 제거
  - 열 레이블 이름을 레이블로 변경(model에서 argument이름을 labels로 지정했기 때문)
  - list 대신 Torch Tensor 형태로 반환하도록 설정


In [187]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1","sentence2","idx"])
tokenized_datasets = tokenized_datasets.rename_column("label","labels")
tokenized_datasets.set_format("torch")

tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

이제 데이터 로더를 정의해보자

In [191]:
from torch.utils.data import DataLoader

train_dataloader=DataLoader(
    dataset = tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator
    )

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=8,
    collate_fn=data_collator
)

잘 생성되는지 빠르게 확인 한 번 해보자

In [192]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 65]),
 'input_ids': torch.Size([8, 65]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 65])}

이제 모델을 불러오자

In [193]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ChangYong/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bi

잘 동작하는지 sample batch를 가지고 테스트해보자

In [195]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.8326, grad_fn=<NllLossBackward>) torch.Size([8, 2])


이제 잘 생성됨을 확인했으니 거의 마지막 과정을 진행!!  
optimizer와 learning rate scheduler가 필요하다!!  
optimizer는 Adam과 유사한 AdamW와 비슷하지만, weight decay regularization이 포함됨

In [196]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

- Scheduler는 기본 lr=5e-5이고, 0으로 linear decay
- 이것을 제대로 정의하려면, epoch과 batch의 개수를 고려하여 decay를 진행해야 한다.

In [198]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

print(num_training_steps)

1377


## The training loop
이제 거의 다 왔다..
GPU를 사용할 경우 model을 gpu에 load한다!

In [200]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

거의 다 됐고, 이제 tqdm 라이브러리를 활용해 교육 상황을 표시줄을 추가하자

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

이 과정은 초반과 유사하게, 어떠한 결과 지표도 설정하지 않아서 출력되지 않는다..   
평가 loop을 추가해보자

## The evaluation loop
- dataset library에서 제공하는 metric 사용
- metric.compute는 이미 봤지만, metric에 add_batch를 활용하여 배치를 축적하고 모든 batch를 얻으면 compute하여 최종 결과를 얻는 방법으로 다음과 같이 구현

In [203]:
from datasets import load_metric

metric = load_metric("glue","mrpc")
model.eval()

for batch in eval_dataloader:
    batch = {k : v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits,dim=-1)
    metric.add_batch(predictions=predictions,references=batch["labels"])
metric.compute()

    

## Supercharge your training loop with 🤗 Accelerate
- 앞서 정의한 training loop은 단일 cpu or gpu에선 잘 작동
- 하지만 accelerate library를 사용하면 multiple GPUs or TPUs를 활성화 가능
- training & validation dataloader를 만드는 것부터 시작하면 다음과 같다

In [ ]:
from datasets import load_dataset
from transformers import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import get_scheduler
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

#load dataset
raw_datasets = load_dataset("glue","mrpc")

#load pretrain object
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

#get tokenized_dataset
def tokenzie_function(example):
    return tokenizer(example["sentence1"],example["sentence2"],truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

#get collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#post-process totenized_datasets
tokenized_datasets = tokenized_datasets.rename_column(["sentence1","sentence2","idx"])
tokenized_datasets.set_format("torch")

#dataloader
train_dataloader = DataLoader(
    dataset = tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    dataset = tokenized_datasets["validation"],
    batch_size=8,
    collate_fn=data_collator
)

#setting device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# optmizer
optimizer = AdamW(model.parameters(), lr=5e-5)

#sheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps
)

progress_bard = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

전체 코드에서 일부 추가!! 및 제거

In [ ]:
# +
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# + 
accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# - 
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# - 
# model.to(device)

# +
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
     train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # -
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        # -
        # loss.backward()
        # +
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

- 추가할 첫 번째 행은 library import 부분
- 추가할 두 번째 줄은 환경을 살펴보고 적절한 분산 설정을 초기화하는 바로 연결 개체를 인스턴스화
-  Accelerator()는 사용자 대신 장치 배치를 처리하므로 장치에 모델을 배치하는 라인을 제거
  - 또는 device 대신 accelerator.device()를 사용해도 무방함
- 그리고 삭제하는 부분들은 device나 batch 처리 관련 부분으로 만약 accelerator.device()를 사용하면 남겨둠
- 하지만 loss.bacward()는 교체!!

---

- train.py에 수정한 전체 코드를 넣으면, 분산 설정에서 실행 할 수 있도록 가능
- 분산 설정에서 테스트하려면 다음 명령을 싱행
  - accelerate config
    - 이걸 실행하면, 몇가지 answer에 답변하라는 prompt가 뜸
  - accelerate launch train.py
    - 이걸 실행하면 distributed training 시작
- notebook에서 실행하려면 맨 아래 다음과 같은 코드 추가
- ```python
   from accelerate import notebook_launcher
   notebook_launcher(training_function)
```